# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Assumptions
- We only consider the totals/national fields, not the values per district/region. Sometimes, the sum of the values per region is a bit higher than the total. We assume that's because some smaller regions are not shown in the data.
    - For this reason, if the *Totals* value is NaN, we cannot just sum up the values of the different regions.
- The fields "National" in SL and Liberia mean the same as the "Totals" field in Guinea, i.e. the Total number of cases in the country.
- When there's a NaN (the value was not filled), we assume that the data was lost, or not written down.
- We are only interested in the Totals/National columns. If a row contains a NaN in this column, we chose to discard the row. This should not be problematic (if the number of NaNs in *Totals* is reasonably low), since we consider the daily **average** per month.
- Since we don't have additionnal information on the data, especially about the meanings of some fields, we decide to compute the new cases separaterly for the confirmed, probable and suspect cases. Same goes for the deaths.

### Process
- first, we import the data and store it separately for each country
- then we rename the fields that we will use (Totals, Description,...) so that they are the same for each country
- we also add two new fields "Country" and "Month" (derived from Date) in each of the 3 dataframes, to simplify the process for later
- We then concatenate the 3 dataframes into a single dataframe, called *full_data*
- Then, to facilitate readability we crop this dataframe (keep only the columns *month*, *Description*, *Totals*, *Country*) and store the result in *full_data_crop*
- We do a couple of sanity checkups, to ensure that the data is well-formated and not erroneous (1 <= month <= 12, no NaN value, etc)

In [ ]:
# imports
import pandas as pd
import os
import numpy as np

In [ ]:
# import data
guinea = []
liberia = []
sl = []
dfs = [guinea, liberia, sl]
i = 0
folders = ["guinea_data", "liberia_data", "sl_data"]
for folder in folders:
    path = DATA_FOLDER + "/ebola/"
    for file in os.listdir(path + folder):
        dfs[i].append(pd.read_csv(path + folder + '/' + file))
    i += 1

In [ ]:
# import Guinea data into DataFrame
guinea = pd.DataFrame()
i = 0
for df in dfs[0]:
    df['Country'] = 'Guinea' # set the new country column
    dfs[0][i] = df
    i += 1
guinea = pd.concat(dfs[0])
guinea.head(2)

In [ ]:
# import liberia data into DataFrame
liberia = pd.DataFrame()
i = 0
for df in dfs[1]:
    df['Country'] = 'Liberia'
    df = df.rename(columns = {'National':'Totals', 'Variable':'Description'})
    dfs[1][i] = df
    i += 1
liberia = pd.concat(dfs[1])
liberia.head(2)

In [ ]:
# import Sierra Leone data into DataFrame
sl = pd.DataFrame()
i = 0
for df in dfs[2]:
    df['Country'] = 'Sierra Leone'
    df = df.rename(columns = {'National':'Totals', 'date' : 'Date', 'variable':'Description'})
    dfs[2][i] = df
    i += 1
sl = pd.concat(dfs[2])
sl.head(2)

In [ ]:
# merge dataframes
full_data = pd.concat([guinea, liberia, sl])

# remove each row whose 'Totals' value is NaN
n_col_before = full_data.shape[0]
full_data = full_data[full_data.Totals.notnull()]
n_col_after = full_data.shape[0]
print("Removed " + str(n_col_before-n_col_after) + " rows out of " + str(n_col_before))

# Specify that the Date column contains datetime objects
full_data.Date = pd.to_datetime(full_data.Date)

# To facilitate readability, we discard all the columns we don't need
full_data_crop = full_data[['Country', 'Description', 'Totals', 'Date']]

In [ ]:
# This is what the dataframe that we'll use to compute the different values looks like so far
full_data_crop.head(2)

In [ ]:
# Names and regexes that we use to process the Description

interests_rgx = ['(new_suspected|New cases of suspects|New Case\/s \(Suspected\))',
             '(new_probable|New cases of probables|New Case\/s \(Probable\))',
             '(new_confirmed|New cases of confirmed|New case\/s \(confirmed\))',
             '(death_suspected|Total deaths of suspects|Total death\/s in suspected cases)',
             '(death_probable|Total deaths of probables|Total death\/s in probable cases)',
             '(death_confirmed|Total deaths of confirmed|Total death\/s in confirmed cases)'
            ]

interests_new_cases = ['New cases (suspected)', 'New cases (probable)', 'New cases (confirmed)']
interests_death = ['Deaths in suspected cases', 'Deaths in probable cases', 'Deaths in confirmed cases']
interests_names = interests_new_cases + interests_death

In [ ]:
# We decided to unstack the Descriptions column in other columns

# uniformize description in needed columns
for i in range(len(interests_rgx)):
    full_data_crop.Description = full_data_crop.Description.str.replace(interests_rgx[i], interests_names[i])

# remove un-needed rows based on Description column
_filter = full_data_crop.Description.isin(interests_names)
full_data_crop = full_data_crop[_filter]

# unstack data
full_data_crop.index = pd.MultiIndex.from_arrays([full_data_crop.Description.values, full_data_crop.Country.values, full_data_crop.Date.values])

# We discovered that there are 3 duplicated lines in the file 2014-10-04 from liberia
# We then had to delete the duplicated rows from the data so that we could unstack it
full_data_crop = full_data_crop[~full_data_crop.index.duplicated(keep='first')]
full_data_crop = full_data_crop.drop(['Description', 'Date', 'Country'], axis=1)

# unstack the Descriptions and remove unneccessary index level
full_data_crop = full_data_crop.unstack(level=0)
full_data_crop.columns = full_data_crop.columns.droplevel()

# convert fields to numeric values
full_data_crop = full_data_crop.apply(pd.to_numeric)

full_data_crop

In [129]:
# put Date and Country back as columns (from the indexes) to make groupbys cleaner
full_data_crop['Date'] = full_data_crop.index.get_level_values(1).values
full_data_crop['Country'] = full_data_crop.index.get_level_values(0).values

x = full_data_crop[full_data_crop.Country == 'Guinea']
y = x.groupby(x.Date.dt.month)
Guinea_Confirmed = (np.max(y['Deaths in confirmed cases']) - np.min(y['Deaths in confirmed cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
Guinea_Probable = (np.max(y['Deaths in probable cases']) - np.min(y['Deaths in probable cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
Guinea_Suspected = (np.max(y['Deaths in suspected cases']) - np.min(y['Deaths in suspected cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
Guinea_New_Confirmed = y['New cases (confirmed)'].mean()
Guinea_New_Probable = y['New cases (probable)'].mean()
Guinea_New_Suspected = y['New cases (suspected)'].mean()
print('Daily average deaths for Guinea')
print(Guinea_Confirmed)
print(Guinea_Probable)
print(Guinea_Suspected)
print(Guinea_New_Confirmed)
print(Guinea_New_Probable)
print(Guinea_New_Suspected)


x = full_data_crop[full_data_crop.Country == 'Liberia']
y = x.groupby(x.Date.dt.month)
Liberia_Confirmed = (np.max(y['Deaths in confirmed cases']) - np.min(y['Deaths in confirmed cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
Liberia_Probable = (np.max(y['Deaths in probable cases']) - np.min(y['Deaths in probable cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
Liberia_Suspected = (np.max(y['Deaths in suspected cases']) - np.min(y['Deaths in suspected cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
print('Daily average deaths for Liberia')
print(Liberia_Confirmed)
print(Liberia_Probable)
print(Liberia_Suspected)

x = full_data_crop[full_data_crop.Country == 'Sierra Leone']
y = x.groupby(x.Date.dt.month)
SL_Confirmed = (np.max(y['Deaths in confirmed cases']) - np.min(y['Deaths in confirmed cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
SL_Probable = (np.max(y['Deaths in probable cases']) - np.min(y['Deaths in probable cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
SL_Suspected = (np.max(y['Deaths in suspected cases']) - np.min(y['Deaths in suspected cases'])).divide((np.max(y['Date']) - np.min(y['Date'])).dt.days)
print('Daily average deaths for Sierra Leone')
print(SL_Confirmed)
print(SL_Probable)
print(SL_Suspected)



In [ ]:
# we assume that the deaths are cumulative (everyday, you get the number of deaths so far, including the deaths that occured
# during previous days). Thus we proceed as follows: 
# - take the max and min number of death for each <Month, Country> pair, take their different 

### remarks

TODO deaths are cumulative, can clone column, shift it and substract

In [ ]:
# test shift
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
   'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])}
df = pd.DataFrame(d)
print(df)
df = df.two.shift(1)
print(df)
assert(df.isnull().sum()==1)
df = df.fillna(0)
print(df)

In [ ]:
# compute the desired values

interests = ['(new_suspected|New cases of suspects|New Case\/s \(Suspected\))',
             '(new_probable|New cases of probables|New Case\/s \(Probable\))',
             '(new_confirmed|New cases of confirmed|New case\/s \(confirmed\))',
             '(death_suspected|Total deaths of suspects|Total death\/s in confirmed cases)',
             '(death_probable|Total deaths of probables|Total death\/s in probable cases)',
             '(death_confirmed|Total deaths of confirmed|Total death\/s in confirmed cases)'
            ]
death_interests = [
            '(death_suspected|Total deaths of suspects|Total death\/s in confirmed cases)',
            '(death_probable|Total deaths of probables|Total death\/s in probable cases)',
            '(death_confirmed|Total deaths of confirmed|Total death\/s in confirmed cases)'
            ]
all_means = []
for interest in interests:
    rows = full_data_crop[full_data_crop.Description.str.match(interest)]
    #print(rows.shape)
    rows.Totals = pd.to_numeric(rows.Totals)
    if(interest in death_interests):
        #print(type(rows))
        total_deaths = rows.Totals.shift(1).fillna(0)
        print('before: ')
        print(rows.Totals[1:10])
        rows.Totals = rows.Totals - total_deaths
        print('after: ')
        print(rows.Totals[1:10])
        
    merge_index = [rows.Month, rows.Country]
    means = rows.groupby(merge_index).agg(np.mean)
    #means = means[['Totals']]
    #assert((rows.Totals >= 0).all()) # assert that the Totals is always >= 0
    
    
    # test
    n_values = rows.groupby(merge_index).agg(['count', np.mean])
    #print(means)
    #print(n_values['Totals'])
    # /test

    means.columns = [interest]
    all_means.append(means)

current = all_means[0]
for mean in all_means[1:]:
    current = pd.merge(current, mean,  left_index=True, right_index=True)
current

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
dfs = []
path = DATA_FOLDER + "/microbiome/"
for file in os.listdir(path):
    if file != 'metadata.xls' :
        dfs.append(pd.read_excel(path + '/' + file , header = None , index_col = 0))
    
Meta = pd.read_excel("Data/microbiome/metadata.xls", index_col = 0)    

In [ ]:
Meta['SAMPLE'].fillna('unknown' , inplace = True )
arrays = [Meta['GROUP'].values, Meta['SAMPLE'].values]
arrays

In [ ]:
merge = pd.concat(dfs , axis = 1)
merge.columns = arrays
merge.fillna('unknown' , inplace = True )
merge

In [ ]:
print(merge.shape)
merge.index.is_unique

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here